# Data Linkage Training Course

We have two datasets which we want to link **practitioner_large_file** & **practitioner_small_file**:

- **practitioner_large_file** contains the variables `IDA`, `Sex`, `Locality`, `Yearbirth`, `Monthbirth`, `Daybirth`, `Name` and a record ID that is contained in the variable `Bident`. In addition, there is a variable `Aident` that contains the record ID from the small file that it is matched to, i.e. we know the true match status.

- **practitioner_small_file** contains the variables `IDB`, `Sex`, `Locality`, `Yearbirth`, `Monthbirth`, `Daybirth`, `Name` and a record ID that is contained in the variable `Aident`. In addition there is a variable, `Bident`, that contains the record ID from the large file that it is matched to, i.e. we know the true match status.

Some of the variables in the **practitioner_small_file** were perturbed to simulate measurement errors. These are called `Sexpert`, `Yearbirthpert`, `Monthbirthpert`, `Daybirthpert`, `Namepert`. `Locality` was not perturbed and we will use this as a blocking variable.

In [5]:
# import Pandas for data manipulation os to read the working directory
import os
import pandas as pd

# Modify the settings so any variable or statement on its own line is displayed
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Widen output display
pd.set_option('display.width', 1000)

def get_file_path():
    # This is the filepath where the datasets and the matchkey file can be found
    file_path = os.getcwd()
    return file_path

# Read in datasets to link. Get the filepath from the command line arguments
filepath = get_file_path()

dfA = pd.read_csv(filepath + '/practitioner_large_file.csv') # 1,000 rows
dfB = pd.read_csv(filepath + '/practitioner_small_file.csv') # 680 rows

# Make sure column types correct
dfA = dfA.astype({"SEX_A": float,
                  "locality_A": float,
                  "yearbirth_A": float,
                  "monthbirth_A": float,
                  "daybirth_A": float,
                  "Name_A": str})
    
dfB = dfB.astype({"SEX_B": float,
                  "locality_B": float,
                  "yearbirth_B": float,
                  "monthbirth_B": float,
                  "daybirth_B": float,
                  "Name_B": str})

Have a look at the data.

In [6]:
# Display the columns headings and first 20 rows
dfA.head(20)
dfB.head(20)

,SEX_A,locality_A,yearbirth_A,monthbirth_A,daybirth_A,Name_A,bident_A,aident_A,ID_A
0,1.0,31.0,1958.0,12.0,3.0,geoff,E373,A1,2051588
1,1.0,31.0,1965.0,1.0,30.0,james,E371,NaN,2047429
2,1.0,70.0,1922.0,7.0,28.0,Michael,E348,NaN,1926422
3,1.0,70.0,1946.0,1.0,NaN,terry,E346,NaN,1907224
4,1.0,70.0,1949.0,9.0,6.0,MATTHEW,E332,A2,1818785
5,1.0,70.0,1961.0,9.0,19.0,micaell,E342,A3,1894191
6,1.0,70.0,1961.0,12.0,3.0,Neil,E341,A4,1887808
7,1.0,70.0,1963.0,11.0,26.0,matthew,E333,NaN,1825535
8,1.0,70.0,1964.0,11.0,28.0,Kornell,E331,A5,1816773
9,1.0,70.0,1967.0,8.0,27.0,richard,E343,NaN,1898234


,SEX_B,locality_B,yearbirth_B,monthbirth_B,daybirth_B,Name_B,namepert_B,yearbirthpert_B,monthbirthpert_B,daybirthpert_B,sexpert_B,aident_B,bident_B,ID_B
0,1.0,7400.0,1975.0,7.0,6.0,alan,alan,75.0,7,6.0,1.0,A245,E925,5196460
1,1.0,7900.0,1975.0,9.0,30.0,JulianDickin,Julian,1076.0,9,30.0,1.0,A266,E20,97397
2,2.0,962.0,1900.0,6.0,NaN,Naomi,Naomi,1900.0,6,NaN,2.0,A377,E621,3563159
3,2.0,6500.0,1905.0,3.0,NaN,debbie,debbie,1905.0,4,NaN,2.0,A544,E903,5115700
4,1.0,3000.0,1905.0,3.0,NaN,Mike,Mike,1905.0,3,NaN,1.0,A95,E291,1576027
5,1.0,7800.0,1907.0,3.0,19.0,peter,deter,1907.0,3,19.0,1.0,A254,E885,5029982
6,2.0,6200.0,1908.0,12.0,20.0,christine,christine,1908.0,12,20.0,2.0,A527,E191,945609
7,1.0,8700.0,1908.0,5.0,NaN,paul,paul,1908.0,5,NaN,1.0,A301,E464,2619977
8,2.0,8300.0,1909.0,11.0,6.0,anita,anita,1909.0,11,6.0,2.0,A622,E255,1343394
9,1.0,188.0,1909.0,5.0,1.0,john,john,1909.0,5,1.0,1.0,A14,E627,3575957


# Step 1: Exact Match using the unperturbed variables

In [ ]:
# Convert name variables in both datasets to uppercase before exact match
dfA['Name_A'] = dfA['Name_A'].str.upper()
dfB['Name_B'] = dfB['Name_B'].str.upper()

# Exact match (inner join - links records from dfA and dfB if they agree on all the 
# variables we are using for matching, in this case sex, locality, year, month and day of birth and name)
# Think of the inner part of a venn diagram.
linked_df = dfA.merge(dfB,
                      left_on = ['SEX_A', 'locality_A', 'yearbirth_A', 'monthbirth_A', 'daybirth_A', 'Name_A'],
                      right_on = ['SEX_B', 'locality_B', 'yearbirth_B', 'monthbirth_B', 'daybirth_B', 'Name_B'],
                      how = 'inner')

# Check the length of the linked dataset
print('Number of records matched: ',len(linked_df))

# Examine the residuals (outer join - gives us the records from dfA and dfB that do not
# agree on all of the matching variables). Think of the outer circles in a venn diagram.
residuals = dfA.merge(dfB,
                      left_on = ['SEX_A', 'locality_A', 'yearbirth_A', 'monthbirth_A', 'daybirth_A', 'Name_A'],
                      right_on = ['SEX_B', 'locality_B', 'yearbirth_B', 'monthbirth_B', 'daybirth_B', 'Name_B'],
                      how = 'outer',
                      indicator = True)

# Check residuals (i.e. records that do not have a match) from dfA
residualsA = residuals[residuals['_merge'] == 'left_only'] 
print('Number of unmatched records in dataframeA: ',len(residualsA))

# Check residuals from dfB
residualsB = residuals[residuals['_merge'] == 'right_only'] 
print('Number of unmatched records in dataframeB: ',len(residualsB))

As expected, all 680 records from the smaller dataset (`dfB`) linked to the larger dataset (`dfA`), as we have used the unperturbed variables.

# Step 2: Exact Match using the perturbed variables of `dfB`

In [ ]:
# Convert peturbed name variable for dfB to uppercase before exact match
dfB['namepert_B'] = dfB['namepert_B'].str.upper()

# Exact match (inner join) using same variables (perturbed)
linked_df2 = dfA.merge(dfB,
                       left_on =  ['SEX_A', 'locality_A', 'yearbirth_A', 'monthbirth_A', 'daybirth_A', 'Name_A'],
                       right_on = ['sexpert_B', 'locality_B', 'yearbirthpert_B', 'monthbirthpert_B', 'daybirthpert_B', 'namepert_B'],
                       how = 'inner')




# Examine the residuals
residuals2 = dfA.merge(dfB,
                       left_on =  ['SEX_A', 'locality_A', 'yearbirth_A', 'monthbirth_A', 'daybirth_A', 'Name_A'],
                       right_on = ['sexpert_B', 'locality_B', 'yearbirthpert_B', 'monthbirthpert_B', 'daybirthpert_B', 'namepert_B'],
                       how = 'outer',
                      indicator = True)

Remember that there were no unmatched records in `dfB` before. How many are there now?

In [ ]:
# Check residuals from dfB
residualsB = residuals2[residuals2['_merge'] == 'right_only'] 
print('Number of unmatched records in dataframeB: ',len(residualsB))

This time, 174 records from the smaller dataset (`dfB`) did not link to a record in the larger dataset (`dfA`) due to errors in the data. Let's have a look at the residuals from `dfB` to see what types of errors caused names not to be matched.

In [ ]:
# View names of clean & perturbed dataset B residuals - errors may have prevented exact match
residualsB[['Name_B', 'namepert_B']]

What kind of errors in the data can you see?

# Step 3: Clean the data by parsing names into two where appropriate and stripping off titles.

Parse the name variables in both datasets. For example 'John Smith' would be split into two variables `name1` and `name2`.

In [ ]:
# Create two new columns name1 and name2, by splitting the name column by the delimiter ' '
# Name2 will be 'None' if there was only one name
# For dfB use namepert instead of name
dfA[['Name1_A', 'Name2_A']] = dfA['Name_A'].str.split(' ',expand=True)
dfB[['Name1pert_B', 'Name2pert_B']] = dfB['namepert_B'].str.split(' ',expand=True)

Delete redundant words. In this case, we only have first names in the file. There are prefixes that appear in the dataset, such as: Mr. Smith or Mrs. Smith. Since we only have one field for name in the dataset, we will delete the redundant word and put the more ‘relevant’ name in  that  field. In this  simple case, the only redundant words in our datasets  are ‘MR’ and  ‘MRS’, so we will simply use a `where` / `if-else` statement in the program.

In [ ]:
# Swap Name 1 and Name 2 if Name 1 is Mr or Mrs
import numpy as np
dfA['Name1_A'], dfA['Name2_A'] = np.where((dfA['Name1_A'] == 'MR') | (dfA['Name1_A'] == 'MRS'), [dfA['Name2_A'], dfA['Name1_A']], [dfA['Name1_A'], dfA['Name2_A']])
dfB['Name1pert_B'], dfB['Name2pert_B'] = np.where((dfB['Name1pert_B'] == 'MR') | (dfB['Name1pert_B'] == 'MRS'), [dfB['Name2pert_B'], dfB['Name1pert_B']], [dfB['Name1pert_B'], dfB['Name2pert_B']])

# Set Name 2 to missing if it is Mr or Mrs
dfA['Name2_A'] = np.where((dfA['Name2_A'] == 'MR') | (dfA['Name2_A'] == 'MRS'), None, dfA['Name2_A'])
dfB['Name2pert_B'] = np.where((dfB['Name2pert_B'] == 'MR') | (dfB['Name2pert_B'] == 'MRS'), None, dfB['Name2pert_B'])

# Have a look at dataframe A to check the changes we've made
dfA[['Name1_A', 'Name2_A']].head(20)


# Exact match (inner join) using the variables (perturbed) after cleaning
linked_df3 = dfA.merge(dfB,
                       left_on =  ['SEX_A', 'locality_A', 'yearbirth_A', 'monthbirth_A', 'daybirth_A', 'Name1_A'],
                       right_on = ['sexpert_B', 'locality_B', 'yearbirthpert_B', 'monthbirthpert_B', 'daybirthpert_B', 'Name1pert_B'],
                       how = 'inner')

# Find the residuals
residuals3 = dfA.merge(dfB,
                       left_on =  ['SEX_A', 'locality_A', 'yearbirth_A', 'monthbirth_A', 'daybirth_A', 'Name1_A'],
                       right_on = ['sexpert_B', 'locality_B', 'yearbirthpert_B', 'monthbirthpert_B', 'daybirthpert_B', 'Name1pert_B'],
                       how = 'outer',
                       indicator = True)

Remember that there were 174 unmatched records in `dfB` before. How many are there now?

In [ ]:
# Check residuals from dfB
residualsB = residuals3[residuals3['_merge'] == 'right_only']
print('Number of unmatched records in dataframeB: ',len(residualsB))

We have improved things a little by cleaning the `name` strings. But there are still 166 unmatched records in `dfB`.

In [ ]:
# Have a look at the residuals - what kind of errors in the data do we still have?
residualsB[['Name_B', 'Name1pert_B']]

# Step 4a: Rule-Based Matching - Relax Name

Until now we have used exact matching. If we relax the rules a little we should find more matches. But beware - we may also introduce some false positives. What happens if we relax the criteria for `name` to match to only insisting that the first three letters of `name` match?

In [ ]:
# Make new columns containing only the first three letters of the name in each dataset
dfA['Short_Name_A'] = dfA.Name1_A.str[0:3]
dfB['Short_Name_B'] = dfB.Name1pert_B.str[0:3]

# Join on the new Short Name variable and sex, locality and date of birth
linked_df4a = dfA.merge(dfB,
                       left_on =  ['SEX_A', 'locality_A', 'yearbirth_A', 'monthbirth_A', 'daybirth_A', 'Short_Name_A'],
                       right_on = ['sexpert_B', 'locality_B', 'yearbirthpert_B', 'monthbirthpert_B', 'daybirthpert_B', 'Short_Name_B'],
                       how = 'inner')



# Find the residuals
residuals4a = dfA.merge(dfB,
                       left_on =  ['SEX_A', 'locality_A', 'yearbirth_A', 'monthbirth_A', 'daybirth_A', 'Short_Name_A'],
                       right_on = ['sexpert_B', 'locality_B', 'yearbirthpert_B', 'monthbirthpert_B', 'daybirthpert_B', 'Short_Name_B'],
                       how = 'outer',
                       indicator = True)

Remember that there were 166 unmatched records in `dfB` before. How many are there now?

In [ ]:
# Count residuals from dfB
residualsB = residuals4a[residuals4a['_merge'] == 'right_only']
print('Number of unmatched records in dataframeB: ',len(residualsB))

Great! Now we only have 114 unmatched records - BUT are all of the matches correct? Check to see if `IDA=IDB` for all
the records in the linked dataset.

In [ ]:
# Add a column 'Match_Status' that has value 1 if the match is correct and 0 otherwise
linked_df4a['Match_Status'] = np.where(linked_df4a['ID_A'] == linked_df4a['ID_B'],1,0)

print('Number of correct matches: ', len(linked_df4a[linked_df4a['Match_Status']==1]))
print('Number of false positives: ', len(linked_df4a[linked_df4a['Match_Status']==0]))

# Examine the false positives
FPs = linked_df4a[linked_df4a['Match_Status']==0]

# View
FPs[['SEX_A', 'sexpert_B',
     'yearbirth_A', 'yearbirthpert_B',
     'monthbirth_A', 'monthbirthpert_B',
     'daybirth_A', 'daybirthpert_B',
     'Name_A', 'namepert_B']]

There are five false positives. Go back to the start of step 4a and change the length of the `Short_Name_A` and `Short_Name_B`. What is the best length of string to use to find the most true positives with the least false positives?

# Step 4b: Rule-Based Matching - Relax Date of Birth

What happens if we relax the criteria for date of birth. Up until now we have insisted that day, month and year all match exactly. Let's see what happens if we let day be a mismatch.

In [ ]:
linked_df4b = dfA.merge(dfB,
                       left_on =  ['SEX_A', 'locality_A', 'monthbirth_A','yearbirth_A',  'Name1_A'],
                       right_on = ['sexpert_B', 'locality_B',  'monthbirthpert_B', 'yearbirthpert_B','Name1pert_B'],
                       how = 'inner')

# Find the residuals
residuals4b = dfA.merge(dfB,
                       left_on =  ['SEX_A', 'locality_A', 'monthbirth_A','yearbirth_A',  'Name1_A'],
                       right_on = ['sexpert_B', 'locality_B',  'monthbirthpert_B', 'yearbirthpert_B','Name1pert_B'],
                       how = 'outer',
                       indicator = True)

How many residuals are there now?

In [ ]:
# Check residuals from dfB
residualsB = residuals4b[residuals4b['_merge'] == 'right_only']
print('Number of unmatched records in dataframeB: ',len(residualsB))

How many of the matches are correct?

In [ ]:
#Check to see if ID_A=ID_B for all the records in the linked dataset
#Add a column 'Match_Status' that has value 1 if the match is correct and 0 otherwise
linked_df4b['Match_Status'] = np.where(linked_df4b['ID_A'] == linked_df4b['ID_B'],1,0)

print('Number of correct matches: ', len(linked_df4b[linked_df4b['Match_Status']==1]))
print('Number of false positives: ', len(linked_df4b[linked_df4b['Match_Status']==0]))

#Examine the false positives
FPs = linked_df4b[linked_df4b['Match_Status']==0]

# View
FPs[['SEX_A', 'sexpert_B',
     'yearbirth_A', 'yearbirthpert_B',
     'monthbirth_A', 'monthbirthpert_B',
     'daybirth_A', 'daybirthpert_B',
     'Name_A', 'namepert_B']]

There is a false positive. Change the variables in the merges to see if we should remove a different, or another, date of birth variable.

# Step 5 - using Levenshtein edit distance

In [ ]:
#The function LD returns the Levenshtein edit distance between two strings
def LD(s, t):
    if s == "":
        return len(t);
    if t == "":
        return len(s);
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1, 
               LD(s[:-1], t[:-1]) + cost])
    return res;


#Test the function by inputting some words of your choice
string1="Rachel"
string2="Rachael"
print('Levenshtein Edit Distance [',string1,',',string2,'] = ',LD(string1, string2))

Note that `[Rachel, Rachael]` and `[Tim, Tom]` both have a `LD` score of 1. But one change in a three letter word is more significant than one change in a six letter word. To account for different name lengths, we'll also calculate a standardised Levenshtein Edit Distance.

In [ ]:
def SLD(s,t):
    standardised = 1-(LD(s,t)/max(len(s),len(t)))
    return standardised;

string1="Rachel"
string2="Rachael"
print('Standardised Levenshtein Edit Distance [',string1,',',string2,'] = ',SLD(string1, string2))

We'll use the Levenshtein distance as part of score based matching in Step 7.

# Step 6: Blocking

First assume we don’t have a blocking variable. We will generate all possible pairs in the two datasets using the perturbed variables.

In [ ]:
# We can do this by creating two columns containing a single value and then join on that value
dfA['join_col'] = 0
dfB['join_col'] = 0

# Produce all possible candidate pairs
CP1 = dfA.merge(dfB,
                on = 'join_col',
                how = 'inner')

`dfA` contains 1,000 records and `dfB` contains 680 records.

In [ ]:
print('There are', len(CP1), 'possible pairs. Of these, 680 (0.1%) are true matches.')

Now we will use the blocking variable `locality`. This isn't perturbed hence we expect to retain all 680 true matches (and also find extra, false matches).

In [ ]:
# produce candidate pairs using blocking variable locality
CP2 = dfA.merge(dfB,
                left_on = 'locality_A',
                right_on = 'locality_B',
                how = 'left')

print('There are', len(CP2), 'possible pairs. Of these, 680 (3.6%) are true matches.')

# Step 7: Score Based Matching

In [ ]:
#Assign weights to the variables used for matching
name_w=5
sex_w=1
day_w=2
month_w=2
year_w=3

CP1['Score1']=(((CP1.Name1_A==CP1.Name1pert_B)*name_w)
              +((CP1.SEX_A==CP1.sexpert_B)*sex_w)
              +(((CP1.daybirth_A==CP1.daybirthpert_B) |(np.isnan(CP1.daybirth_A) & np.isnan(CP1.daybirthpert_B)))*day_w)
              +(((CP1.monthbirth_A==CP1.monthbirthpert_B) | (np.isnan(CP1.monthbirth_A) & np.isnan(CP1.monthbirthpert_B)))*month_w)
              +(((CP1.yearbirth_A==CP1.yearbirthpert_B) | (np.isnan(CP1.yearbirth_A) & np.isnan(CP1.yearbirthpert_B)))*year_w))             

#This will tell us how many candidate pairs scored each of 0 (complete mismatch) to 13 (exact match)
CP1.Score1.value_counts(sort=False)

There are 514 exact matches as expected - same as the number of residials after `name` cleaning and exact matching.

In [ ]:
CP2['Score2']=(((CP2.Name1_A==CP2.Name1pert_B)*name_w)
              +((CP2.SEX_A==CP2.sexpert_B)*sex_w)
              +(((CP2.daybirth_A==CP2.daybirthpert_B) |(np.isnan(CP2.daybirth_A) & np.isnan(CP2.daybirthpert_B)))*day_w)
              +(((CP2.monthbirth_A==CP2.monthbirthpert_B) | (np.isnan(CP2.monthbirth_A) & np.isnan(CP2.monthbirthpert_B)))*month_w)
              +(((CP2.yearbirth_A==CP2.yearbirthpert_B) | (np.isnan(CP2.yearbirth_A) & np.isnan(CP2.yearbirthpert_B)))*year_w))                
           

#This will tell us how many candidate pairs scored each of 0 (complete mismatch) to 13 (exact match)
CP2.Score2.value_counts(sort=False)

In [ ]:
#Set the threshold for accepting as a match
Threshold=10
Matches = CP2[CP2['Score2'] >= Threshold]
print('With threshold', Threshold, 'there are ',len(Matches), 'matches.')
Matches.loc[:,'Match_Status']=np.where(Matches['ID_A'] == Matches['ID_B'],1,0)   # ignore warning

# TP and FP
print('Number of correct matches: ', len(Matches[Matches['Match_Status']==1]))
print('Number of false positives: ', len(Matches[Matches['Match_Status']==0]))

What do the different scores tell us? You can alter the weights and threshold until you are happy that you have maximised true positives and minimised false positives.

# Step 8: Score Based Matching with partial scores

Here we will use the Levenshtein Edit Distance to award a partial score for names that partially match. So names like Rachel and Rachael will score `0.857*name_w` rather than zero.

In [ ]:
# Replace NaN with blank space
CP2['Name1_A'] = np.where(CP2['Name1_A'].isnull(), '', CP2['Name1_A'])
CP2['Name1pert_B'] = np.where(CP2['Name1pert_B'].isnull(), '', CP2['Name1pert_B'])

# Apply edit distance to pairs (warning - this takes about 12 minutes)
CP2['SLD']=CP2.apply(lambda x: SLD(x['Name1_A'], x['Name1pert_B']), axis=1)

# Calculate score
CP2['Score2']=((CP2.SLD*name_w)
              +((CP2.SEX_A==CP2.SEX_B)*sex_w)
              +((CP2.daybirth_A==CP2.daybirthpert_B)*day_w)
              +((CP2.monthbirth_A==CP2.monthbirthpert_B)*month_w)
              +((CP2.yearbirth_A==CP2.yearbirthpert_B)*year_w))           

In [ ]:
Threshold=10
Matches2 = CP2[CP2['Score2']>=Threshold]
print('With threshold', Threshold ,'there are',len(Matches2), 'matches.')
Matches2.loc[:,'Match_Status']=np.where(Matches2['ID_A'] == Matches2['ID_B'],1,0)

# TP and FP
print('Number of correct matches: ', len(Matches2[Matches2['Match_Status']==1]))
print('Number of false positives: ', len(Matches2[Matches2['Match_Status']==0]))

Change the `Threshold` to see if you can make the false postive rate smaller.